In [1]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

providing ../data/raw/2022_place_canvas_history-000000000050.csv ...
../data/raw/2022_place_canvas_history-000000000050.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5852af43-7981-45cd-8ea1-26610a53cbb7;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 85ms :: artifacts dl 3ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



<div style="color:lightgreen">
Eine zweite zu betrachtende Situation ist, dass Moderatoren auch mehrere Pixel auf einmal ändern können.

```
SELECT user_id, COUNT(x) --es reicht aus die x Koordinate zu zählen
FROM dataFrame
WHERE --Wie könnte die Where-Clause angegeben werden, dass nur ein bestimmter Zeitpunkt betrachtet wird?
HAVING COUNT(x) > 1
GROUP BY user_id;
```


Vorschlag Sebastian:
<div style="color:lightblue">(Sebastian): Gruppieren nach t-Spalte und dann gucken wo count(t) > 1 ?</div>
</div>

<div style="color:lightgreen">

Mein Vorschlag:

Ist es nicht besser direkt nach Nutzer und Zeit zu gruppieren? Dann hat man für jeden Nutzer direkt angezeigt, wie oft zu einem Zeitpunkt Pixel platziert wurden?
Dann wäre auch die Bereinigung der Daten effektiver als oben.

Hier der Versuch:
</div>

In [2]:
from src.features.feature_functions import group_dataframe_by_time_and_user
import pyspark.sql.functions as F

timeUserGrouped = group_dataframe_by_time_and_user(dataFrame)
timeUserGrouped.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- t: long (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [3]:
timeUserGrouped = timeUserGrouped.withColumnRenamed('count','time_count')

timeUserGrouped = timeUserGrouped.filter(timeUserGrouped.time_count > 1)
timeUserGrouped.show()

+--------------------+----+----------+--------------------+--------------------+-------------------------+
|             user_id|   t|time_count|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+--------------------+----+----------+--------------------+--------------------+-------------------------+
|X8gpDfO6+Z87SKKTy...|2974|         8|[1685, 904, 932, ...|[228, 1865, 1850,...|     [#FFD635, #000000...|
|AxBTmhBMbr1sbDw/2...|2344|         6|[1403, 777, 1741,...|[2, 8, 10, 2, 8, 10]|     [#FFA800, #FFA800...|
|DIECtVqoJqBlL6A0C...|1200|         6|[932, 920, 933, 9...|[1757, 1832, 1838...|     [#FFFFFF, #BE0039...|
|AxBTmhBMbr1sbDw/2...|1726|         6|[655, 612, 1575, ...|[1896, 1917, 7, 1...|     [#B44AC0, #E4ABFF...|
|8GviOLIfRTg2/rkNm...|2039|         4|[911, 1758, 911, ...|[101, 900, 101, 900]|     [#D4D7D9, #000000...|
|YcEvoWbE9DFR5hqKD...|3179|         4|[1308, 1308, 1308...|[877, 876, 877, 876]|     [#000000, #000000...|
|uekG383QBM2sRK0En...|2791|         4

In [8]:
grouped = timeUserGrouped.dropDuplicates(['user_id'])
grouped.show()

+--------------------+----+----------+---------------+---------------+-------------------------+
|             user_id|   t|time_count|collect_list(x)|collect_list(y)|collect_list(pixel_color)|
+--------------------+----+----------+---------------+---------------+-------------------------+
|++++KubvRsRI2sZI3...|1827|         2|   [1434, 1434]|     [360, 360]|       [#9C6926, #9C6926]|
|++0nCmngtZ68vbg6p...| 734|         2|     [757, 757]|     [592, 592]|       [#000000, #000000]|
|++3Ki0p8wbp0g7xQv...|3424|         2|   [1975, 1975]|       [92, 92]|       [#FFFFFF, #FFFFFF]|
|++3gcvcX6sgZwlgO7...|1574|         2|     [982, 982]|   [1528, 1528]|       [#000000, #000000]|
|++3l0k1HSXKiAsHkH...|1721|         2|     [664, 664]|   [1240, 1240]|       [#9C6926, #9C6926]|
|++4/asfR6j/RbsvIF...|2741|         2|     [467, 467]|     [937, 937]|       [#FFFFFF, #FFFFFF]|
|++4PiR2T1Y0vPxBnK...|3505|         2|     [205, 205]|   [1852, 1852]|       [#000000, #000000]|
|++5bQtHNJxwF1lYE2...| 349|   

In [9]:
print(grouped.count())

22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/06/05 18:17:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


733458


In [4]:
row = dataFrame.count('user_id')
print(f'Anzahl der herkömmlichen Zeilen: {row}')

row_sel = timeUserGrouped.count('user_id')
print(f'Anzahl der angeblichen Moderatoren: {row_sel}')

TypeError: count() takes 1 positional argument but 2 were given

<div style="color:lightgreen">
Vllt doch besser nur über Zeit zu gruppieren?
</div>

In [4]:
from src.features.feature_functions import group_dataframe_by_time

timeGrouped = group_dataframe_by_time(dataFrame)
timeGrouped.printSchema()

root
 |-- t: long (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [5]:
timeGrouped = timeGrouped.withColumnRenamed('count','time_count')
timeGrouped.printSchema()

root
 |-- t: long (nullable = true)
 |-- time_count: long (nullable = false)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [6]:
timeGrouped = timeGrouped.filter(timeGrouped.time_count > 1)
timeGrouped.show()

+---+----------+---------------------+--------------------+--------------------+-------------------------+
|  t|time_count|collect_list(user_id)|     collect_list(x)|     collect_list(y)|collect_list(pixel_color)|
+---+----------+---------------------+--------------------+--------------------+-------------------------+
|502|      1372| [fl3fslGUutRFVv5s...|[1803, 1307, 1073...|[828, 537, 180, 1...|     [#000000, #FFFFFF...|
|503|      1352| [KhKhrQXoREif0iKG...|[453, 1813, 1812,...|[1216, 820, 814, ...|     [#000000, #FFFFFF...|
|506|      1352| [RtdCr10DU/ryVoyP...|[862, 1794, 1580,...|[807, 743, 1500, ...|     [#FF4500, #51E9F4...|
|518|      1336| [0ri9DdoyY1cUGgPO...|[1813, 539, 918, ...|[810, 1240, 575, ...|     [#FFFFFF, #DE107F...|
|510|      1336| [SuvkvnbkdYwMBLGl...|[1538, 1146, 1812...|[507, 551, 815, 1...|     [#D4D7D9, #FFD635...|
|800|      1328| [vLy9enKCuqE5Lfz5...|[939, 1795, 71, 2...|[1640, 949, 1717,...|     [#FFA800, #9C6926...|
|665|      1328| [/Z19GH2yLv1t/Rwn...

<div style="color:lightgreen">
Jz muss für jeden Zeitpunkt die Userliste betrachtet werden.

Dabei muss geschaut werden, welcher User mehrmals vorkommt.

D.h. Vorkommen zählen.
</div>

<div style="color:#b0e0e6">
Alle Spalten bis auf 't' und 'collect_list(user_id)' können entfernt werden, da unwichtig.
</div>

In [7]:
timeDroped = timeGrouped.drop('time_count').drop('collect_list(x)').drop('collect_list(y)').drop('collect_list(pixel_color)')
timeDroped.printSchema()

root
 |-- t: long (nullable = true)
 |-- collect_list(user_id): array (nullable = false)
 |    |-- element: string (containsNull = false)



In [8]:
timeDroped.show()

+---+---------------------+
|  t|collect_list(user_id)|
+---+---------------------+
|502| [fl3fslGUutRFVv5s...|
|506| [RtdCr10DU/ryVoyP...|
|503| [KhKhrQXoREif0iKG...|
|510| [SuvkvnbkdYwMBLGl...|
|518| [0ri9DdoyY1cUGgPO...|
|800| [vLy9enKCuqE5Lfz5...|
|665| [/Z19GH2yLv1t/Rwn...|
|512| [OiXUUFiS09e43mbt...|
|517| [kO8v5t9qQcVGH2jc...|
|509| [47cBtZB9Io6HDvkx...|
|511| [8X81sGoyIyFjlMGA...|
|504| [jNvbu72zK1ym3mbn...|
|496| [ln0Wl8R3QfmA+vHi...|
|816| [gDoibmO3IFsdZG4o...|
|508| [/ovdrfnI5wHVb10l...|
|692| [VwAHZ5/Lrh5ehY3k...|
|514| [S0deEoFnPuCIDPtC...|
|505| [31jd2vAVrOBHCNUF...|
|974| [oTkkJ2uY7S8RVXDZ...|
|673| [0zdlmGBfhD1unCE9...|
+---+---------------------+
only showing top 20 rows



In [9]:
collected = timeDroped.collect()
count = 0
moderator = []

for row in collected:
    time,users = list(row)
    for user in users:
        for comp in users:
            if (user == comp): 
                count +1
            if(user in moderator) : 
                break #in äußere Spalte zurückkehren
            if(count >= 2) : 
                moderator.append(user)
                break #in äußere Spalte zurückkehren
        count = 0
print(moderator)   

[]


<div style="color:#b0e0e6">
Es ergibt sich eine leere Liste, ist das so korrekt?
Denn das würde bedeuten, es gäbe in der einen Stunde keine Moderatoren...
(Siehe auch Anmerkung unten)
</div>

In [12]:
dataFrame.orderBy(['user_id'], ascending = [True]).show()

+--------------------+----+----+----+-----------+
|             user_id|   x|   y|   t|pixel_color|
+--------------------+----+----+----+-----------+
|++++KubvRsRI2sZI3...|1434| 360|1827|    #9C6926|
|++++KubvRsRI2sZI3...|1434| 360|1827|    #9C6926|
|+++GYtxEzILnCHvgI...|1122| 535|3059|    #6D482F|
|+++GYtxEzILnCHvgI...|1122| 535|3059|    #6D482F|
|+++L0Cq2acDZLU86l...|  84|1814| 852|    #FFFFFF|
|+++L0Cq2acDZLU86l...|  84|1814| 852|    #FFFFFF|
|+++ypsBxRdwrVDGfj...| 286|1087|2280|    #FFFFFF|
|+++ypsBxRdwrVDGfj...| 286|1087|2280|    #FFFFFF|
|++/5WaPt9reXoCE3/...| 667| 507|2258|    #FFFFFF|
|++/5WaPt9reXoCE3/...| 667| 507|2258|    #FFFFFF|
|++/a8YPlilhzOTJEW...| 926|1038|2995|    #3690EA|
|++/a8YPlilhzOTJEW...| 934|1056|1427|    #3690EA|
|++/a8YPlilhzOTJEW...| 934|1056|1427|    #3690EA|
|++/a8YPlilhzOTJEW...| 974|1049|1087|    #FFFFFF|
|++/a8YPlilhzOTJEW...| 926|1038|2995|    #3690EA|
|++/a8YPlilhzOTJEW...| 974|1049|1087|    #FFFFFF|
|++/iKk3F3ZlK4sJxa...|1917| 907|2174|    #9C6926|


<div style="color:#b0e0e6">
Warum gibt's bei manchen Usern Dopplungen der selben Zeit in Kombo mit den selben Koordinaten und bei manchen nicht?
Deutet das auf Moderatoren hin?
Evtl. müsste man den Datensatz selbst im Original nochmal anschauen.

Was ich mich auch frage ist, ob die Aufspaltung in x und y Koordinaten so korrekt ist,
da Moderatoren x1,x2,y1,y2 Angaben in der Spalte haben und nicht x,y....
Also ob die Anpassung des Datensatzes, wie Sebastian es vorbereitet hat, so passend ist mit den Moderatoren?
</div>